In [1]:
# python
import sys
import importlib
# columnar analysis
from coffea import processor
from coffea.nanoevents import NanoAODSchema
import awkward as ak
from dask.distributed import Client
# local
sidm_path = str(sys.path[0]).split("/sidm")[0]
if sidm_path not in sys.path: sys.path.insert(1, sidm_path)
from sidm.tools import utilities, sidm_processor, scaleout
# always reload local modules to pick up changes during development
importlib.reload(utilities)
importlib.reload(sidm_processor)
importlib.reload(scaleout)

<module 'sidm.tools.scaleout' from '/home/cms-jovyan/SIDM/sidm/tools/scaleout.py'>

In [2]:
client = scaleout.make_dask_client("tls://localhost:8786")
client

/home/cms-jovyan/SIDM/sidm/tools/scaleout.py:12: UserWarning: Registering a `SchedulerPlugin` as a worker plugin is not allowed, registering as a scheduler plugin instead. To register as a worker plugin, inherit from `WorkerPlugin`.
  client.register_worker_plugin(PipInstall(packages=dependencies, pip_options=["--upgrade"]))


Connection method: Direct,
Dashboard: /user/bryan.cardwell@cern.ch/proxy/8787/status,
Comm: tls://192.168.235.112:8786,Workers: 0
Dashboard: /user/bryan.cardwell@cern.ch/proxy/8787/status,Total threads: 0
Started: 1 hour ago,Total memory: 0 B


In [3]:
samples = [
    '2Mu2E_500GeV_5p0GeV_8p0mm',
]
fileset = utilities.make_fileset(samples, "llpNanoAOD_v2", max_files=3, location_cfg="signal_2mu2e_v10.yaml")

p = sidm_processor.SidmProcessor(
    ["2mu2e"],
    ["base"],
)

# test if processor is serializable
import coffea.util as coffea_util
coffea_util.save(p, "processor.coffea")
print(coffea_util.load("processor.coffea"))

runner = processor.Runner(
    executor=processor.DaskExecutor(client=client),
    schema=NanoAODSchema,
    maxchunks=1,
    #skipbadfiles=True
)

output = runner.run(fileset, treename="Events", processor_instance=p)

/opt/conda/lib/python3.9/site-packages/dask/dataframe/_pyarrow_compat.py:17: FutureWarning: Minimal version of pyarrow will soon be increased to 14.0.1. You are using 12.0.0. Please consider upgrading.
  warnings.warn(


Exception: Failed processing file: WorkItem(dataset='2Mu2E_500GeV_5p0GeV_8p0mm', filename='root://xcache//store/group/lpcmetx/SIDM/ULSignalSamples/2018_v10/BsTo2DpTo2Mu2e/CutDecayFalse_SIDM_BsTo2DpTo2Mu2e_MBs-500_MDp-5p0_ctau-8p0_v3/LLPnanoAODv2/CutDecayFalse_SIDM_BsTo2DpTo2Mu2e_MBs-500_MDp-5p0_ctau-8p0_v3_part-11.root', treename='Events', entrystart=0, entrystop=4424, fileuuid=b'\x14\x9e\x94\x94\xbf\x87\x11\xef\x83\xa6\x8f\x15\xe6\x9b\xbe\xef', usermeta={})

In [ ]:
out = output["out"]
print(out[samples[0]]['cutflow'])

In [ ]:
# test that ouput is accumulatable
from coffea.processor import accumulate
double_out = accumulate([output["out"], output["out"]])

In [ ]:
out[samples[0]]["cutflow"]["2mu2e"].print_table()
out[samples[0]]["cutflow"]["2mu2e"].print_table(fraction=True)

In [ ]:
double_out[samples[0]]["cutflow"]["2mu2e"].print_table()
double_out[samples[0]]["cutflow"]["2mu2e"].print_table(fraction=True)